In [9]:
from dash import Dash, html,  Input, Output ,dcc #dash core components
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql

In [10]:
user = "bigdata"
password = 'Bigdata123!!'
host = '192.168.56.101'
port = '3306'
db = 'SampleDB'
connect_script = f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'
engine = create_engine(connect_script)
conn = engine.connect()

In [3]:
df = px.data.gapminder()

In [4]:
df.dtypes

country       object
continent     object
year           int64
lifeExp      float64
pop            int64
gdpPercap    float64
iso_alpha     object
iso_num        int64
dtype: object

In [5]:
stmt = '''
create table world2
(country varchar(50),
 continent varchar(50),
 year int,
 lifeExp float,
 pop int,
 gdpPercap float,
 iso_alpha varchar(50),
 iso_num int
)
'''
conn.execute(text(stmt))

In [6]:
qry=('''
select *
from world2
''')
pd.read_sql(qry,conn)

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num


In [7]:
insert_stmt = '''
insert into world2(
country,
continent,
year,
lifeExp,
pop,
gdpPercap,
iso_alpha,
iso_num
)
values(
:country,
:continent,
:year,
:lifeExp,
:pop,
:gdpPercap,
:iso_alpha,
:iso_num
)
'''

for dt in df.to_dict('records'):
    conn.execute(text(insert_stmt),**dt)

In [18]:
pd.read_sql(qry,conn)

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.101,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.421,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.450,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.039,ZWE,716


In [34]:
app = Dash(__name__) #앱을 띄운다.
app.layout = html.Div(
    [
        html.H3("국가의 인구변화"),
        dcc.Dropdown(id="dropdown-nation",
                    options=[{"label":country+"국가","value":country}for country in df.country.unique()]),
        html.Hr(),
        dcc.Graph(id='dcc-graph',
                  figure = fig),
        
    ]
)

@app.callback(
    Output("dcc-graph","figure"),
    Input("dropdown-nation","value")
)
def update_graph(nation):
    stmt = f"select * from world2 where country = '{nation}'"
    df = pd.read_sql(stmt,conn)
    fig = px.bar(df,x="year",y="pop")
    return fig

app.run_server(host='192.168.56.101',port=7777)

Dash is running on http://192.168.56.101:7777/

Dash is running on http://192.168.56.101:7777/

Dash is running on http://192.168.56.101:7777/

Dash is running on http://192.168.56.101:7777/

Dash is running on http://192.168.56.101:7777/

Dash is running on http://192.168.56.101:7777/

Dash is running on http://192.168.56.101:7777/

Dash is running on http://192.168.56.101:7777/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.56.101:7777/ (Press CTRL+C to quit)
192.168.56.1 - - [18/Feb/2022 15:17:30] "GET / HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 15:17:30] "GET /_dash-layout HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 15:17:30] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 15:17:30] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 15:17:30] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 15:17:30] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 15:17:31] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 15:17:32] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 15:17:32] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 15:17:33] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 15:17:34]

In [33]:
df = px.data.gapminder()
dt = df.loc[df.country == "Afghanistan",:]
fig = px.bar(df,x='year',y='pop')